# Modelo: Random Forest

El objetivo principal de este cuaderno es:
* Entrenar un modelo Random Forest, que es un ensamble de árboles de decisión.
* Evaluar su desempeño utilizando el conjunto de validación.
* Obtener métricas finales (accuracy, matriz de confusión, classification report).

**Nota:** Este modelo es más complejo que Regresión Logística pero puede capturar patrones no lineales de forma efectiva.

### Importaciones


En esta sección cargamos todas las librerías necesarias para entrenamiento, evaluación y generación de predicciones.


In [2]:
import os, json
import zipfile
import pandas as pd
import numpy as np
from google.colab import userdata

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier

### Traemos los datos desde kaggle

In [5]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key
assert user and key, "Faltan los secretos KAGGLE_USERNAME/KAGGLE_KEY"
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

"kaggle" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
El sistema no puede encontrar la ruta especificada.
"wc" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [6]:
df_train = pd.read_csv("train.csv")
df_test  = pd.read_csv("test.csv")
df_sample = pd.read_csv("submission_example.csv")

df_train.head(3)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264


### Preprocesado de la Data

Para el preprocesado usare la funcion propuesta para la entrega #2 con unas pequeñas adaptaciones

In [7]:
def make_scaler(kind: str):
    kind = (kind or 'standard').lower()
    if kind in ('standard', 'std', 'zscore'):
        return StandardScaler(with_mean=False)  # compatible con matrices dispersas
    else:
        return StandardScaler(with_mean=False)


def build_preprocessor_v2(
    df: pd.DataFrame,
    y_col: str = 'RENDIMIENTO_GLOBAL',
    id_col: str = 'ID',
    num_impute: str = 'median',
    cat_impute: str = 'most_frequent',
    encode_categorical: str = 'onehot',
    ordinal_maps: dict = None,
    scale_numeric: str = 'standard'
):
    data = df.copy()
    y = data.pop(y_col) if y_col in data.columns else None

    # Quitar columnas no predictoras
    for col in [id_col, 'Y_NUMERIC']:
        if col in data.columns:
            data = data.drop(columns=col)

    # Tipos
    num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
    cat_cols = data.select_dtypes(include=['object']).columns.tolist()

    # Ordinal vs OneHot
    ordinal_maps = ordinal_maps or {}
    ordinal_cols = [c for c in cat_cols if c in ordinal_maps]
    onehot_cols  = [c for c in cat_cols if c not in ordinal_maps]

    # Pipelines
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=num_impute)),
        ('scaler', make_scaler(scale_numeric))
    ])

    onehot_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=True))
    ])

    ordinal_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('ordinal', OrdinalEncoder(
            categories=[ordinal_maps[c] for c in ordinal_cols],
            handle_unknown='use_encoded_value', unknown_value=-1
        ))
    ]) if ordinal_cols else 'drop'

    preprocessor = ColumnTransformer(
        transformers=[
            ('num',    num_pipe,    num_cols),
            ('onehot', onehot_pipe, onehot_cols),
            ('ordinal', ordinal_pipe, ordinal_cols)
        ],
        remainder='drop',
        n_jobs=1
    )
    return preprocessor, data, y

In [8]:
ordinal_maps = {
  'E_VALORMATRICULAUNIVERSIDAD': [
      'NO PAGO MATRICULA',
      'MENOS DE 500 MIL',
      'ENTRE 500 MIL Y MENOS DE 1 MILLON',
      'ENTRE 1 MILLON Y MENOS DE 2.5 MILLONES',
      'ENTRE 2.5 MILLONES Y MENOS DE 4 MILLONES',
      'ENTRE 4 MILLONES Y MENOS DE 5.5 MILLONES',
      'ENTRE 5.5 MILLONES Y MENOS DE 7 MILLONES',
      'MAS DE 7 MILLONES'
  ],
  'E_HORASSEMANATRABAJA': [
      '0',
      'MENOS DE 10 HORAS',
      'ENTRE 11 Y 20 HORAS',
      'ENTRE 21 Y 30 HORAS',
      'MAS DE 30 HORAS'
  ],
  'F_ESTRATOVIVIENDA': [
      'SIN ESTRATO', 'ESTRATO 1', 'ESTRATO 2', 'ESTRATO 3', 'ESTRATO 4', 'ESTRATO 5', 'ESTRATO 6'
  ],
  'F_EDUCACIONPADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
    ],
    'F_EDUCACIONMADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
  ]
}

In [9]:
preproc, X_df, y = build_preprocessor_v2(
    df_train,
    y_col='RENDIMIENTO_GLOBAL',
    id_col='ID',
    num_impute='median',
    cat_impute='most_frequent',
    encode_categorical='onehot',
    ordinal_maps=ordinal_maps,
    scale_numeric='standard'
)

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_df, y, test_size=0.2, stratify=y, random_state=42
)

### Entrenamiento del modelo Random Forest


Entrenamos un Random Forest con hiperparámetros iniciales.
Este modelo es más complejo que Regresión Logística pero puede capturar patrones no lineales y capturar interacciones entre variables de forma automática.
**Importante:** El entrenamiento puede tardar varios minutos debido a la cantidad de árboles y la profundidad máxima configurada.


In [22]:
rf = Pipeline(steps=[
    ('prep', preproc),
    ('clf', RandomForestClassifier(
        n_estimators=600,
        max_depth=70,
        min_samples_split=5,
        min_samples_leaf=1,
        n_jobs=-1,
        random_state=42,
        verbose=1
    ))
])
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 28.4min finished


,steps,"[('prep', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('onehot', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Evaluación del modelo


Incluimos:
* accuracy
* matriz de confusión
* classification report

Estas métricas permiten visualizar el comportamiento del modelo y entender sus fortalezas y debilidades.

**Análisis de viabilidad:** Aunque el modelo Random Forest ofrece un `accuracy` muy bueno, el tiempo de entrenamiento (~30 minutos en máquina local, más en Colab) lo hace poco viable para ciclos iterativos rápidos o entornos con recursos limitados.


In [23]:
preds_rf = rf.predict(X_valid)
acc_rf  = accuracy_score(y_valid, preds_rf)
f1_rf  = f1_score(y_valid, preds_rf, average='macro')

print(f'Accuracy: {acc_rf:.4f} | F1-macro: {f1_rf:.4f}')
print('\nReporte:\n', classification_report(y_valid, preds_rf))
print('\nMatriz de confusión:\n', confusion_matrix(y_valid, preds_rf, labels=sorted(y.unique())))

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    2.5s
[Parallel(n_jobs=12)]: Done 600 out of 600 | elapsed:    3.5s finished


Accuracy: 0.3920 | F1-macro: 0.3806

Reporte:
               precision    recall  f1-score   support

        alto       0.49      0.59      0.53     35124
        bajo       0.42      0.49      0.45     34597
  medio-alto       0.29      0.23      0.26     34324
  medio-bajo       0.31      0.26      0.28     34455

    accuracy                           0.39    138500
   macro avg       0.38      0.39      0.38    138500
weighted avg       0.38      0.39      0.38    138500


Matriz de confusión:
 [[20582  3851  6281  4410]
 [ 4168 16782  5561  8086]
 [10963  7868  7893  7600]
 [ 6704 11520  7196  9035]]
